<a href="https://colab.research.google.com/github/sfansaria/CrimePredictionUsingLSTM-RandomForestClassifier-RandomForestRegressor-ARIMA/blob/main/Crime_rf_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load the drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import the Libraries

In [3]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import math



from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error



In [4]:
def formatNumber(n):
    return '{:,}'.format(n).replace(","," ")


def highlight_diag(df):
  a = np.full(df.shape, '',dtype='<U24')
  np.fill_diagonal(a, 'background-color: yellow')
  return pd.DataFrame(a, index=df.index, columns=df.columns)


#Import the Dataset

In [5]:
crime_df = pd.read_csv('/content/drive/My Drive/ml_datasets/Crime_Data_from_2020_to_Present.csv')

In [6]:
print("Number of entries:", formatNumber(crime_df.shape[0]))

Number of entries: 901 357


#data info

In [7]:
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901357 entries, 0 to 901356
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           901357 non-null  int64  
 1   Date Rptd       901357 non-null  object 
 2   DATE OCC        901357 non-null  object 
 3   TIME OCC        901357 non-null  int64  
 4   AREA            901357 non-null  int64  
 5   AREA NAME       901357 non-null  object 
 6   Rpt Dist No     901357 non-null  int64  
 7   Part 1-2        901357 non-null  int64  
 8   Crm Cd          901357 non-null  int64  
 9   Crm Cd Desc     901357 non-null  object 
 10  Mocodes         775767 non-null  object 
 11  Vict Age        901357 non-null  int64  
 12  Vict Sex        781966 non-null  object 
 13  Vict Descent    781958 non-null  object 
 14  Premis Cd       901347 non-null  float64
 15  Premis Desc     900807 non-null  object 
 16  Weapon Used Cd  312268 non-null  float64
 17  Weapon Des

#Drop the attributes

In [8]:
drop_attributes = ['DATE OCC', 'TIME OCC', 'Rpt Dist No', 'Part 1-2', 'Mocodes', 'Vict Descent', 'Premis Cd', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4']

crime_df_new = crime_df.drop(columns=drop_attributes)

crime_df_new.drop(crime_df_new[crime_df_new['Vict Sex'] == "N"].index, inplace=True)

crime_df_new.drop(crime_df_new[crime_df_new['Vict Sex'] == "H"].index, inplace=True)

crime_dff = crime_df_new
(crime_dff.columns)


Index(['DR_NO', 'Date Rptd', 'AREA', 'AREA NAME', 'Crm Cd', 'Crm Cd Desc',
       'Vict Age', 'Vict Sex', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc',
       'LOCATION', 'Cross Street', 'LAT', 'LON'],
      dtype='object')

In [9]:
crime_dff.head(1)

,DR_NO,Date Rptd,AREA,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Used Cd,Weapon Desc,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,7,Wilshire,510,VEHICLE - STOLEN,0,M,STREET,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506


#Models

#Classification
#Random Forest

In [10]:
crime_dff.head(5)

,DR_NO,Date Rptd,AREA,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Used Cd,Weapon Desc,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,7,Wilshire,510,VEHICLE - STOLEN,0,M,STREET,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,1,Central,330,BURGLARY FROM VEHICLE,47,M,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,3,Southwest,480,BIKE - STOLEN,19,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,9,Van Nuys,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19,M,CLOTHING STORE,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,6,Hollywood,354,THEFT OF IDENTITY,28,M,SIDEWALK,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [11]:
drop2_attributes = ['DR_NO', 'Date Rptd', 'AREA NAME', 'Crm Cd', 'Premis Desc', 'Weapon Desc','LOCATION','Cross Street']
crime_dff_1 = crime_dff.drop(columns=drop2_attributes)
crime_dff_1 = crime_dff_1.dropna()
crime_dff_1.head(1)

,AREA,Crm Cd Desc,Vict Age,Vict Sex,Weapon Used Cd,LAT,LON
9,19,BATTERY - SIMPLE ASSAULT,26,M,400.0,34.2336,-118.4535


* Move the crime description to the last column

In [12]:
crime_dff_1 = crime_dff_1[['AREA','Vict Age','Vict Sex','LAT','LON','Weapon Used Cd','Crm Cd Desc']]
crime_dff_1.head()

,AREA,Vict Age,Vict Sex,LAT,LON,Weapon Used Cd,Crm Cd Desc
9,19,26,M,34.2336,-118.4535,400.0,BATTERY - SIMPLE ASSAULT
11,19,8,F,34.2285,-118.4258,400.0,SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO...
12,2,7,F,34.0868,-118.2991,400.0,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...
13,4,8,F,34.0784,-118.1936,400.0,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...
19,7,31,F,34.0452,-118.3351,307.0,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"


#Top 3 crimes

In [13]:
top3_crimes = crime_dff_1['Crm Cd Desc'].value_counts().head(3).index
top3_crimes

Index(['BATTERY - SIMPLE ASSAULT',
       'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
       'INTIMATE PARTNER - SIMPLE ASSAULT'],
      dtype='object', name='Crm Cd Desc')

In [14]:
crime_dff_1 = crime_dff_1[crime_dff_1['Crm Cd Desc'].isin(top3_crimes)]
crime_dff_1.head()

,AREA,Vict Age,Vict Sex,LAT,LON,Weapon Used Cd,Crm Cd Desc
9,19,26,M,34.2336,-118.4535,400.0,BATTERY - SIMPLE ASSAULT
19,7,31,F,34.0452,-118.3351,307.0,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"
31,8,24,F,34.0474,-118.4635,400.0,BATTERY - SIMPLE ASSAULT
66,8,42,F,34.0596,-118.4233,400.0,INTIMATE PARTNER - SIMPLE ASSAULT
94,3,25,F,34.0055,-118.2915,400.0,INTIMATE PARTNER - SIMPLE ASSAULT


#Normalize the remaining labels of the dataset to convert them into numerical values.

In [15]:
crime_dff_1.loc[crime_dff_1['Vict Sex'] =='F', 'Vict Sex'] = 1
crime_dff_1.loc[crime_dff_1['Vict Sex'] =='M', 'Vict Sex'] = 0
crime_dff_1.loc[crime_dff_1['Vict Sex'] =='X', 'Vict Sex'] = 2
crime_dff_1

,AREA,Vict Age,Vict Sex,LAT,LON,Weapon Used Cd,Crm Cd Desc
9,19,26,0,34.2336,-118.4535,400.0,BATTERY - SIMPLE ASSAULT
19,7,31,1,34.0452,-118.3351,307.0,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"
31,8,24,1,34.0474,-118.4635,400.0,BATTERY - SIMPLE ASSAULT
66,8,42,1,34.0596,-118.4233,400.0,INTIMATE PARTNER - SIMPLE ASSAULT
94,3,25,1,34.0055,-118.2915,400.0,INTIMATE PARTNER - SIMPLE ASSAULT
...,...,...,...,...,...,...,...
901326,16,8,1,34.2374,-118.3964,400.0,BATTERY - SIMPLE ASSAULT
901327,5,59,0,33.7868,-118.2658,200.0,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"
901329,4,58,1,34.0932,-118.1615,400.0,BATTERY - SIMPLE ASSAULT
901350,1,35,0,34.0626,-118.2445,400.0,INTIMATE PARTNER - SIMPLE ASSAULT


#Normalization of classification categories as numeric values
* Creating the dependent variable class

In [16]:
factor = pd.factorize(crime_dff_1['Crm Cd Desc'])
crime_dff_1['Crm Cd Desc'] = factor[0]
definitions = factor[1]
print(factor[0])
print(factor[1])
print(definitions)

[0 1 0 ... 0 2 1]
Index(['BATTERY - SIMPLE ASSAULT',
       'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
       'INTIMATE PARTNER - SIMPLE ASSAULT'],
      dtype='object')
Index(['BATTERY - SIMPLE ASSAULT',
       'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
       'INTIMATE PARTNER - SIMPLE ASSAULT'],
      dtype='object')


<ipython-input-16-8dc670a5009b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_dff_1['Crm Cd Desc'] = factor[0]


In [17]:
crime_dff_1

,AREA,Vict Age,Vict Sex,LAT,LON,Weapon Used Cd,Crm Cd Desc
9,19,26,0,34.2336,-118.4535,400.0,0
19,7,31,1,34.0452,-118.3351,307.0,1
31,8,24,1,34.0474,-118.4635,400.0,0
66,8,42,1,34.0596,-118.4233,400.0,2
94,3,25,1,34.0055,-118.2915,400.0,2
...,...,...,...,...,...,...,...
901326,16,8,1,34.2374,-118.3964,400.0,0
901327,5,59,0,33.7868,-118.2658,200.0,1
901329,4,58,1,34.0932,-118.1615,400.0,0
901350,1,35,0,34.0626,-118.2445,400.0,2


#Splitting the dataset into dependent and independent variables.

In [18]:
X = crime_dff_1.iloc[:,0:6].values
y = crime_dff_1.iloc[:,6].values

print('The independent features set: ')
print(X[:5,:])
print('The dependent variables: ')
print(y[:5])

The independent features set: 
[[19 26 0 34.2336 -118.4535 400.0]
 [7 31 1 34.0452 -118.3351 307.0]
 [8 24 1 34.0474 -118.4635 400.0]
 [8 42 1 34.0596 -118.4233 400.0]
 [3 25 1 34.0055 -118.2915 400.0]]
The dependent variables: 
[0 1 0 2 2]


#Split the data into train and test data

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


#Run the Random Forest Algorithm

In [20]:
classifier = RandomForestClassifier(n_estimators=200, criterion='gini')
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

#Prediction using the Random Forest model

In [21]:
y_pred = classifier.predict(X_test)

print('Confusion Matrix : \n',confusion_matrix(y_test, y_pred))
print('Classification Report : \n',classification_report(y_test, y_pred))
print('Accuracy Score : \n',accuracy_score(y_test, y_pred))


Confusion Matrix : 
 [[15186  1375  4787]
 [ 2286 12628   504]
 [ 5746   485  7318]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.65      0.71      0.68     21348
           1       0.87      0.82      0.84     15418
           2       0.58      0.54      0.56     13549

    accuracy                           0.70     50315
   macro avg       0.70      0.69      0.70     50315
weighted avg       0.70      0.70      0.70     50315

Accuracy Score : 
 0.6982410811885124


In [22]:
reverse_factor = dict(zip(range(len(definitions)),definitions))
reverse_factor

{0: 'BATTERY - SIMPLE ASSAULT',
 1: 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 2: 'INTIMATE PARTNER - SIMPLE ASSAULT'}

In [23]:
y_test_backup = y_test


#Reverse the normalization

In [24]:
y_test = np.vectorize(reverse_factor.get)(y_test)
y_pred = np.vectorize(reverse_factor.get)(y_pred)

#Confusion Matrix between Actual values and Predicted Values

In [25]:
cross = pd.crosstab(y_test, y_pred, rownames=['Actual Crime'], colnames = ['Predicted Crime'])

In [26]:
cross.style.apply(highlight_diag, axis=None)

Predicted Crime,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",BATTERY - SIMPLE ASSAULT,INTIMATE PARTNER - SIMPLE ASSAULT
Actual Crime,,,
"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",12628,2286,504
BATTERY - SIMPLE ASSAULT,1375,15186,4787
INTIMATE PARTNER - SIMPLE ASSAULT,485,5746,7318


* The Accuracy is about 70% but the loader is very important for the algorithm, so try to improve the algorithm by reducing the number of the features.
*Using Random Forest classifier, select the feature with threshold having a value >= 0.15

In [27]:
sfm = SelectFromModel(classifier, threshold=0.15)
sfm.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(n_estimators=200),
                threshold=0.15)

In [28]:
sfm.get_support(indices=True)

array([1, 3, 4, 5])

#Transform the data into a new dataset containing all the important features.

In [29]:
X_imp_train = sfm.transform(X_train)
X_imp_test = sfm.transform(X_test)

In [30]:
#Create a new Random Forest Classifier for the most important features

imp_classifier = RandomForestClassifier(n_estimators=200,criterion='gini')

#Train the new classifier on the new dataset containing the important features
imp_classifier.fit(X_imp_train, y_train)

RandomForestClassifier(n_estimators=200)

In [31]:
y_test = y_test_backup

In [32]:
#Predict the test results
y_pred_imp = imp_classifier.predict(X_imp_test)

print('Confusion Matrix : \n', confusion_matrix(y_test, y_pred_imp))
print('Classification Report : \n', classification_report(y_test, y_pred_imp))
print('Accuracy Score : \n', accuracy_score(y_test, y_pred_imp))

Confusion Matrix : 
 [[14841  1408  5099]
 [ 2201 12509   708]
 [ 6441   604  6504]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.63      0.70      0.66     21348
           1       0.86      0.81      0.84     15418
           2       0.53      0.48      0.50     13549

    accuracy                           0.67     50315
   macro avg       0.67      0.66      0.67     50315
weighted avg       0.67      0.67      0.67     50315

Accuracy Score : 
 0.6728411010633012


In [33]:
#Reverse the normalization
y_test = np.vectorize(reverse_factor.get)(y_test)
y_pred = np.vectorize(reverse_factor.get)(y_pred_imp)

In [34]:
#Make a confusion matrix

cross = pd.crosstab(y_test, y_pred, rownames=['Actual Crimes'], colnames=['Predicted Crimes'])
cross.style.apply(highlight_diag, axis=None)

Predicted Crimes,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",BATTERY - SIMPLE ASSAULT,INTIMATE PARTNER - SIMPLE ASSAULT
Actual Crimes,,,
"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",12509,2201,708
BATTERY - SIMPLE ASSAULT,1408,14841,5099
INTIMATE PARTNER - SIMPLE ASSAULT,604,6441,6504
